In [10]:
from numba import jit, vectorize, float32, float64
import numpy as np
from scipy import spatial, integrate, optimize
from matplotlib import pyplot as plt
from SmoothingLength import HsmlIter

normdict = {1: 4./3, 3: 8./np.pi, 2: 40/(7*np.pi)}

@vectorize([float32(float32), float64(float64)])
def Kernel(q):
    if q <= 0.5:
        return 1 - 6*q**2 + 6*q**3
    elif q <= 1.0:
        return 2 * (1-q)**3
    else: return 0.0
    
@vectorize([float32(float32), float64(float64)])
def DKernel(q):
    if q <= 0.5:
        return -12*q + 18*q*q
    elif q <= 1.0:
        return -6 * (1-q)**2
    else: return 0.0

@vectorize([float32(float32), float64(float64)])
def Kernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (1 - 6*q**2 + 6*q**3)
    elif q <= 1.0:
        return 2.5464790894703255 * 2 * (1-q)**3
    else: return 0.0
    
@vectorize([float32(float32), float64(float64)])
def DKernel3D(q):
    if q <= 0.5:
        return 2.5464790894703255 * (-12*q + 18*q**2)
    elif q <= 1.0:
        return -2.5464790894703255 * 6 * (1-q)**2
    else: return 0.0
    
@jit
def HydroForce(x, m, f, K, dK, rho, h, ngb, ngbdist):
    A = 1.0
    N, dim = x.shape
    Nngb = ngbdist.shape[1]
    a = np.zeros_like(x)
    for i in xrange(N):
        ngbi = ngb[i]
        xngb = x[ngbi]
        mngb = m[ngbi]
        hngb = h[ngbi]
        rngb = ngbdist[i]
        fngb = f[ngbi]
        Ki = K[i]
        dKi = dK[i]
        hi = h[i]
        fi = f[i]
        rhoi = rho[i]
        for j in xrange(1,Nngb):
            atot = mngb[j] * (fi * A * rhoi**(gamma-2.0) * dKi[j] / hi**(dim+1)
            + fngb[j] * A * rho[ngbi[j]]**(gamma-2.0) * norm*DKernel(rngb[j]/h[ngbi[j]]) / h[ngbi[j]]**(dim+1))
            #atot = mngb[j] * (rhoi**(gamma-2.0) + rho[ngbi[j]]**(gamma-2.0)) * dKi[j] / hi**4.0
            for k in xrange(dim):
                dx1 = xngb[j,k] - xngb[0,k]
                dx2 = -np.sign(dx1) * (1.0 - np.abs(dx1))
                if np.abs(dx1) < np.abs(dx2):
                    dx = dx1
                else:
                    dx = dx2
                a[i,k] += atot*dx/rngb[j]
                
    return a


@jit
def KernelSum(f,r,h):
    return np.sum(f*norm*Kernel(r/h)/h**dim)

In [11]:
N=  100
#x = np.random.rand(N,1)**0.5
#x = (np.arange(32)/32.)
#x = x + 0.001*np.sin(2*np.pi*x)
#y = np.arange(32)/32.
#x,y,z = np.meshgrid(x,y,y)
#x = np.c_[x.flatten(),y.flatten(),z.flatten()]
#x = (x)%1.0
dim = 1
norm = normdict[dim]
dtmin = (5./3)**0.5 / N /10

T = 1./(5./3)**0.5
nsteps = int(T/dtmin+1)
print nsteps
dt = T/nsteps
m = np.ones(N)/N
gamma = 5./3
x = np.array([np.arange(0,N)/float(N),]).T

x += 0.001*np.sin(2*np.pi*x)
x = x % 1.0
#tree = spatial.cKDTree(x,boxsize=1.0)
des_ngb = 8
v = np.zeros_like(x)
tree = spatial.cKDTree(x, boxsize=1.)
ngbdist, ngb = tree.query(x,des_ngb)
h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
rho0 = m * des_ngb / (2*h)

@jit
def HForce(x, m, h):
    N, dim = x.shape
    tree = spatial.cKDTree(x, boxsize=1.)
    ngbdist, ngb = tree.query(x, des_ngb)
    h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=dim)
    for i in xrange(N):
        rho = m[i] * des_ngb / (2*h)
        fi = 0
        q = ngbdist[i]/h[i]
        K = norm*Kernel(q)
        dK = norm*Kernel(q)
        for j in xrange(des_ngb):
            fi += -m[ngb[i,j]] * h[i]**-(dim+1) * (dK * q )

601


In [12]:
tree = spatial.cKDTree(x, boxsize=1.)
ngbdist, ngb = tree.query(x,des_ngb)
h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
rho = m * des_ngb / (2*h)
q = ngbdist/h[:,np.newaxis]
K = norm*Kernel(q)
dK = norm*DKernel(q)
FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-(dim+1) * (dK *q + dim*K)),axis=1))**-1
F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)

In [13]:
e1 = []
e2 = []
#plt.scatter(x, -np.gradient(rho**gamma)/np.gradient(x[:,0]) / rho * 100.*dt)
for i in xrange(nsteps*2):
    x += v * dt + F * dt**2 / 2
    x[:,0] = x[:,0]%1.0%1.0
    v += F * dt/2
    tree = spatial.cKDTree(x, boxsize=1.)
    ngbdist, ngb = tree.query(x,des_ngb)
    h = HsmlIter(ngb,ngbdist, error_norm=1e-12,dim=1)
    rho = m * des_ngb / (2*h)
    q = ngbdist/h[:,np.newaxis]
    K = norm*Kernel(q)
    dK = norm*DKernel(q)
    FI = (1 + h / (3*rho) * np.sum((-m[:,np.newaxis]*h[:,np.newaxis]**-(dim+1) * (dK *q + dim*K)),axis=1))**-1
    F = HydroForce(x,m,FI, K, dK, rho, h, ngb, ngbdist)
    v += F * dt/2
    e1.append(np.sum(0.5*m*v**2))
    e2.append(np.sum(rho**(5./3) * 3./2 * m / 2 / h))
    #e.append(np.abs(np.sum(F,axis=0)))
    #e.append(np.sum(0.5*m*v**2) + np.sum(3./2 * rho**(5./3)))
#plt.scatter(np.arange(100), np.array(e) - e[0])
#plt.yscale('log')

plt.plot((np.array(e1) + np.array(e2))/np.array(e2))
plt.show()
print (rho-rho0).std()
#plt.yscale('log')
#plt.ylim(0.1, 10)
#plt.scatter(x[:,0],np.gradient(rho**gamma)/np.gradient(x[:,0]) / rho, color='blue')

#if i % (nsteps/10)==0:
    #if i%10==0:
    #    plt.scatter(x[:,0],v[:,0])
    #
#plt.scatter(x[:,0][(x[:,1]==0.0)*(x[:,2]==0.0)], )
        #plt.show()

7.69255331935e-06


In [149]:
-1.85820065e-18 % 1.0

1.0

In [ ]:
KernelSum(m[ngb[0]], ngbdist[0], h[0])/ rho[0]

In [ ]:
plt.loglog(np.logspace(2,7,5), (392e-6,5e-3,64e-3,0.917,13.7))
plt.loglog(np.logspace(2,7,5), 392e-6*np.logspace(2,7,5)*np.log10(np.logspace(2,7,5)/100))
plt.show()

In [ ]:
optimize.curve_fit(lambda x, a, b: a*x + b, np.linspace(2,7,5), np.log10((392e-6,5e-3,64e-3,0.917,13.7)))

In [ ]:
plt.plot(np.linspace(0,1,100), DKernel3D(np.linspace(0,1,100))); plt.show()

In [ ]:
(10**6/10**2)**2